# Define vibrational coordinates and kinetic-energy operator for OCS

Selecting appropriate vibrational __coordinates__ is crucial for accelerating variational calculations by improving the convergence of the __basis set__.

Examples of some commonly used vibrational coordinates for triatomic molecules.

| Coordinate system | Definition | 
|:-:|:-:|
|Valence-bond coordinates | <img src="assets/images/ocs_valence.png" width="300" />|
|Jacobi coordinates |  <img src="assets/images/ocs_jacobi.png" width="300" />|

Note: the orientation of the Cartesian axes $x$, $y$, and $z$ in the molecule does not really matter, as long as molecular rotation and rotational-vibrational coupling (Coriolis coupling) are disregarded.

## Start by loading necessary modules and functions

- `com`: shifts Cartesian coordianates of atoms to the centre of mass.
- `Gmat`: computes kinetic-energy G-matrix.
- `masses`: masses of atoms in the order 'C', 'O', 'S'.
- `jax`: library for computing derivatives of functions; will be necessary for computing the kinetic-energy operator.
- `jax.numpy` or `jnp`: numpy functions that can be differentiated using `jax`. Use it as standard numpy.

In [1]:
from kinetic import com, Gmat, Gmat_batch
from molecule import masses
import jax
from jax import numpy as jnp
from jax import config
import numpy as np

config.update("jax_enable_x64", True)

## Code Cartesian coordinates of atoms as function of **valence-bond** coordinates

 <img src="assets/images/ocs_valence.png" width="200" />

In [2]:
@com
def valence_to_cartesian(valence_coords):
    r_CO, r_CS, alpha = valence_coords
    return jnp.array(
        [
            [0.0, 0.0, 0.0],  # C
            [-r_CO * jnp.cos(alpha / 2), 0.0, -r_CO * jnp.sin(alpha / 2)],  # O
            [-r_CS * jnp.cos(alpha / 2), 0.0, r_CS * jnp.sin(alpha / 2)],  # S
        ],
        dtype=jnp.float64,
    )


def cartesian_to_valence(xyz):
    n_CO = xyz[1] - xyz[0]
    n_CS = xyz[2] - xyz[0]
    r_CO = jnp.linalg.norm(n_CO)
    r_CS = jnp.linalg.norm(n_CS)
    alpha = jnp.arccos(jnp.dot(n_CO, n_CS) / (r_CO * r_CS))
    return jnp.array([r_CO, r_CS, alpha])

# vectorized versions of functions
cartesian_to_valence_batch = jax.vmap(cartesian_to_valence, in_axes=(0,))
valence_to_cartesian_batch = jax.vmap(valence_to_cartesian, in_axes=(0,))

## Code Cartesian coordinates of atoms as function of **Jacobi** coordinates

<img src="assets/images/ocs_jacobi.png" width="200" />

In [3]:
@com
def jacobi_to_cartesian(jacobi_coords):
    r_S, r_CO, theta = jacobi_coords
    mass_C, mass_O, mass_S = masses
    z_C = 0
    z_O = r_CO
    z_com = (z_C * mass_C + z_O * mass_O) / (mass_C + mass_O)
    return jnp.array(
        [
            [0.0, 0.0, 0.0],  # C
            [0.0, 0.0, r_CO],  # O
            [r_S * jnp.sin(theta), 0.0, z_com + r_S * jnp.cos(theta)],  # S
        ],
        dtype=jnp.float64,
    )


def cartesian_to_jacobi(xyz):
    n1 = xyz[1] - xyz[0]
    r_CO = jnp.linalg.norm(n1)
    com = jnp.dot(masses[:2], xyz[:2, :]) / jnp.sum(jnp.array(masses[:2]))
    n2 = xyz[2] - com
    r_S = jnp.linalg.norm(n2)
    theta = jnp.arccos(jnp.dot(n1, n2) / (r_CO * r_S))
    return jnp.array([r_S, r_CO, theta])


# vectorized versions of functions
cartesian_to_jacobi_batch = jax.vmap(cartesian_to_jacobi, in_axes=(0,))
jacobi_to_cartesian_batch = jax.vmap(jacobi_to_cartesian, in_axes=(0,))

## Details of kinetic-energy operator construction

- KEO, expressed in Cartesian coordinates, for a molecule consisting of $i=1..N$ atoms in the laboratory frame:
$$
\hat{T} = -\frac{1}{2}\sum_{i=1}^N\frac{1}{m_i}\sum_{\alpha=x,y,z}\frac{\partial}{\partial r_{i,\alpha}}\frac{\partial}{\partial r_{i,\alpha}}.
$$

- Introduce a coordiante transformation from the laboratory frame to molecular frame, distinguishing $3N-6$ vibrational, 3 rotational, and 3 translational coordinates:
$$
\boldsymbol{\xi} = \{q_1, q_2, ... q_{3N-6},\phi,\theta,\chi,X_\text{CM},Y_\text{CM},Z_\text{CM}\}.
$$

- Rewrite KEO from Cartesian to $\boldsymbol{\xi}$ coordinates:
$$
\hat{T} = -\frac{1}{2}\sum_{i=1}^N\frac{1}{m_i}\sum_{\alpha=x,y,z}\frac{\partial}{\partial r_{i,\alpha}}\frac{\partial}{\partial r_{i,\alpha}} = 
-\frac{1}{2}\sum_{i=1}^N\frac{1}{m_i}\sum_{\alpha=x,y,z}\left(\sum_l\frac{\partial }{\partial \xi_l}\frac{\partial \xi_l}{\partial r_{i,\alpha}}\right)
\left(\sum_m\frac{\partial\xi_m}{\partial r_{i,\alpha}}\frac{\partial}{\partial \xi_m}\right) \approx \\
\approx -\frac{1}{2}\sum_l\sum_m\frac{\partial }{\partial \xi_l}
\underbrace{\left(
\sum_{i=1}^N\frac{1}{m_i}\sum_{\alpha=x,y,z}\frac{\partial \xi_l}{\partial r_{i,\alpha}}
\frac{\partial\xi_m}{\partial r_{i,\alpha}}
\right)}_{G_{l,m}}
\frac{\partial}{\partial \xi_m} =
-\frac{1}{2}\sum_l\sum_m\frac{\partial }{\partial \xi_l}
G_{l,m}
\frac{\partial}{\partial \xi_m}.
$$

- The kinetic-energy $G$ matrix is calculated as:
$$
G_{l,m} = \sum_{i=1}^N\frac{1}{m_i}\sum_{\alpha=x,y,z}\frac{\partial \xi_l}{\partial r_{i,\alpha}}
\frac{\partial\xi_m}{\partial r_{i,\alpha}}
$$

$$
g_{l,m} = \sum_{i=1}^N m_i\sum_{\alpha=x,y,z}\frac{\partial r_{i,\alpha}}{\partial \xi_l}
\frac{\partial r_{i,\alpha}}{\partial \xi_m}
$$

$$
\mathbf{G} = \mathbf{g}^{-1}
$$

- The derivatives of Cartesian coordinates $r_{i,\alpha}$ wrt to $\xi_l$ coordinates can be easily to calculated as:
$$
\frac{\partial r_{i,\alpha}}{\partial \xi_l} = \left\{\begin{array}{ll}
\delta_{\alpha,l} & \text{COM~translation},~\xi_l\in \{X_\text{CM},Y_\text{CM},Z_\text{CM}\} \\
\sum_\gamma\varepsilon_{\alpha l\gamma}r_{i,\gamma} & \text{rotation},~\xi_l\in\{\phi,\theta,\chi\} \\
\partial r_{i,\alpha}/\partial q_l & \text{vibration},~\xi_l\in\{q_1,q_2,...,q_{3N-6}\}
\end{array}\right.
$$
-----

The computation of derivatives of Cartesian coordinates with respect to vibrational coordinates,
$\partial r_{i,\alpha}/\partial q_l$, is accomplished through automatic differentiation
of a user-defined function $r_{i,\alpha} = f(q_1,q_2,...,q_{3N-6})$.
This function accepts vibrational coordinates $(q_1,q_2,...,q_{3N-6})$
as input and returns the $r_{i,\alpha}$ Cartesian coordinates of atoms.
See, for example functions such as `jacobi_to_cartesian` and `valence_to_cartesian`.

The calculation of the $G$-matrix is done by the `kinetic.Gmat` function,
which requires an array of vibrational coordinate values $(q_1, q_2, ..., q_{3N-6})$
along with the function $f(q_1, q_2, ..., q_{3N-6})\rightarrow r_{i,\alpha}$.


In [4]:
print(f"Test serial computation of G-matrix ...")
# define three values of Jacobi coordinates, r_S, r_CO, theta
coords = np.array([1.0, 1.0, np.pi])

G = Gmat(coords, jacobi_to_cartesian)
print("G.shape:", G.shape)
print("G vibrational:\n", G[:3, :3])

# batch-version of Gmat
coords = np.abs(np.random.random((100, 3)))
print(f"Test computation of G-matrix on batch of {len(coords)} points ...")
G = Gmat_batch(coords, jacobi_to_cartesian)
print("G.shape:", G.shape)

Test serial computation of G-matrix ...
G.shape: (9, 9)
G vibrational:
 [[ 2.25885729e+00  1.07628682e-17 -3.59498837e-32]
 [ 1.07628682e-17  4.91747847e+00 -2.05710026e-32]
 [-3.59498837e-32 -2.05710026e-32  7.17633576e+00]]
Test computation of G-matrix on batch of 100 points ...
G.shape: (100, 9, 9)
